In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import cv2
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

In [3]:

train_dir = "E:/VS/Project/DERMNET/train"
test_dir = "E:/VS/Project/DERMNET/test"

In [4]:

label_names = sorted(os.listdir(train_dir))
label_map = {label: index for index, label in enumerate(label_names)}
print("Classes found:", label_names)

Classes found: ['Acne and Rosacea Photos', 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions', 'Atopic Dermatitis Photos', 'Bullous Disease Photos', 'Cellulitis Impetigo and other Bacterial Infections', 'Eczema Photos', 'Exanthems and Drug Eruptions', 'Hair Loss Photos Alopecia and other Hair Diseases', 'Herpes HPV and other STDs Photos', 'Light Diseases and Disorders of Pigmentation', 'Lupus and other Connective Tissue diseases', 'Melanoma Skin Cancer Nevi and Moles', 'Nail Fungus and other Nail Disease', 'Poison Ivy Photos and other Contact Dermatitis', 'Psoriasis pictures Lichen Planus and related diseases', 'Scabies Lyme Disease and other Infestations and Bites', 'Seborrheic Keratoses and other Benign Tumors', 'Systemic Disease', 'Tinea Ringworm Candidiasis and other Fungal Infections', 'Urticaria Hives', 'Vascular Tumors', 'Vasculitis Photos', 'Warts Molluscum and other Viral Infections']


In [5]:

def load_data(data_dir, img_size=(128, 128)):
    images = []
    labels = []

    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for img_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, img_name)
                img = cv2.imread(img_path)  # Read in color (BGR)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
                    img = cv2.resize(img, img_size)
                    images.append(img)
                    if label in label_map:
                        labels.append(label_map[label])
                else:
                    print(f"Warning: Could not load {img_path}")

    images = np.array(images, dtype="float32") / 255.0
    labels = to_categorical(np.array(labels), num_classes=len(label_names))
    return images, labels

In [6]:
print("Train dir exists:", os.path.exists(train_dir))
print("Subfolders in train_dir:", os.listdir(train_dir))

Train dir exists: True
Subfolders in train_dir: ['Acne and Rosacea Photos', 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions', 'Atopic Dermatitis Photos', 'Bullous Disease Photos', 'Cellulitis Impetigo and other Bacterial Infections', 'Eczema Photos', 'Exanthems and Drug Eruptions', 'Hair Loss Photos Alopecia and other Hair Diseases', 'Herpes HPV and other STDs Photos', 'Light Diseases and Disorders of Pigmentation', 'Lupus and other Connective Tissue diseases', 'Melanoma Skin Cancer Nevi and Moles', 'Nail Fungus and other Nail Disease', 'Poison Ivy Photos and other Contact Dermatitis', 'Psoriasis pictures Lichen Planus and related diseases', 'Scabies Lyme Disease and other Infestations and Bites', 'Seborrheic Keratoses and other Benign Tumors', 'Systemic Disease', 'Tinea Ringworm Candidiasis and other Fungal Infections', 'Urticaria Hives', 'Vascular Tumors', 'Vasculitis Photos', 'Warts Molluscum and other Viral Infections']


In [7]:

train_images, train_labels = load_data(train_dir)
test_images, test_labels = load_data(test_dir)

In [8]:

train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42
)

In [9]:

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [10]:

model = models.Sequential([
    layers.Input(shape=(128, 128, 3)),

    layers.Conv2D(32, (3, 3), activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(256, (3, 3), activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(len(label_names), activation="softmax")
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [11]:

early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.5, min_lr=1e-7, verbose=1)
model_checkpoint = ModelCheckpoint("best_dermnet_model.keras", monitor="val_loss", save_best_only=True, verbose=1)

In [12]:

history = model.fit(
    datagen.flow(train_images, train_labels, batch_size=32),
    validation_data=(val_images, val_labels),
    epochs=50,
    callbacks=[early_stopping, lr_reduction, model_checkpoint],
    verbose=1
)

C:\Users\Naveen Singh Rawat\AppData\Roaming\Python\Python313\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
389/389 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1047 - loss: 4.9799
Epoch 1: val_loss improved from None to 4.44988, saving model to best_dermnet_model.keras
389/389 ━━━━━━━━━━━━━━━━━━━━ 655s 2s/step - accuracy: 0.1206 - loss: 4.5005 - val_accuracy: 0.0900 - val_loss: 4.4499 - learning_rate: 1.0000e-04
Epoch 2/50
389/389 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1474 - loss: 4.1772
Epoch 2: val_loss improved from 4.44988 to 3.98799, saving model to best_dermnet_model.keras
389/389 ━━━━━━━━━━━━━━━━━━━━ 425s 1s/step - accuracy: 0.1479 - loss: 4.1672 - val_accuracy: 0.1880 - val_loss: 3.9880 - learning_rate: 1.0000e-04
Epoch 3/50
389/389 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1567 - loss: 4.1022
Epoch 3: val_loss improved from 3.98799 to 3.92194, saving model to best_dermnet_model.keras
389/389 ━━━━━━━━━━━━━━━━━━━━ 451s 1s/step - accuracy: 0.1605 - loss: 4.0856 - val_accuracy: 0.2114 - val_loss: 3.9219 - learning_rate: 1.0000e-04
Epoch 4/50
389/389 ━━━━━━━━━━━━

In [53]:
import json

with open("dermnet_training_history.json", "w") as f:
    json.dump(history.history, f)

In [61]:
test_images, test_labels = load_data("E:/VS/Project/DERMNET/test")


In [62]:

import pickle

# Save model
model.save("dermnet_skin_disease_model.keras")
# Save training history
with open("dermnet_training_history.pkl", "wb") as f:
    pickle.dump(history.history, f)
# Later when running webcam
model = load_model("dermnet_skin_disease_model.keras")

In [63]:
import os
print(os.path.getsize("dermnet_training_history.pkl"))  # Should be > 0
print("Saved keys:", history.history.keys())

2346
Saved keys: dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss', 'learning_rate'])


In [64]:
from tensorflow.keras.models import load_model

In [65]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

Test Accuracy: 0.3348
Test Loss: 2.6377
